In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 


Accessing the website and extracting the div containing the HREF

In [3]:
base_url = "https://www.automobile.tn/fr/occasion/"
page_number = 1
href=[]

while True:
    url = f"{base_url}{page_number}"
    print("Fetching URL:", url)  
    response = requests.get(url)
   

    
    if response.status_code != 200 or page_number==293 :
        break  

    soup = BeautifulSoup(response.content, "html.parser")
    car_announcements = soup.find_all("div", class_="occasion-item")

    for announcement in car_announcements:
        
        href.append(announcement.find("a")["href"])
        print(href[page_number-1])
        

    page_number += 1








Fetching URL: https://www.automobile.tn/fr/occasion/1
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
/fr/occasion/volkswagen/golf-7/96070
Fetching URL: https://www.automobile.tn/fr/occasion/2
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/classe-a/94020
/fr/occasion/mercedes-benz/c

Creating Lists to contain the data

In [4]:

Brand = []
Type = []
Manufacture_year = []
Mileage = []
Price = []
Body_type = []
Fuel = []
Horse_Power = []
Gear_box = []
Transmission = []
Color = []
Governorate = []
Address = []


Function to request the html of each link provided by the href list 

In [5]:
def extract(href):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 OPR/101.0.0.0"
    }
    url = f"https://www.automobile.tn{href}"  
    r = requests.get(url, headers=headers)  
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

Extracting the data

In [6]:
for link in href:
    
    

    div_elements = extract(link).find_all('div', class_='content-container container-lg')
   
    for div in div_elements:
        h3_element = div.find('h3', class_='page-title')
        if h3_element:
            brand, type = h3_element.text.split(' ', 1)
            Brand.append(brand)
            Type.append(type)

        price_span =div.find('div', class_='small-price').find('span')
        Price.append(price_span.text.strip())   
        li_elements = div.find('ul', class_='list-unstyled').find_all('li')
        for li in li_elements:
            label = li.find('label')
            span = li.find('span')
            
            if label and span:
                label_text = label.text.strip()
                span_text = span.text.strip()
                
                if label_text == 'Kilométrage :':
                    Mileage.append(span_text)
                elif label_text == 'Année :':
                    Manufacture_year.append(span_text)
                elif label_text == 'Gouvernorat :':
                    Governorate.append(span_text)

        address_element = div.find('div', class_='description').find('p')
        Address.append( address_element.text.strip())

    tables = div.find('div', class_='technical-details').find_all('table', attrs={'data-index': 'main'})
    for table in tables:
        thead = table.find('thead')
        if thead:
            tbody = table.find('tbody')
            if "Carrosserie" in thead.get_text():
                Body_type.append( tbody.get_text().strip())
            elif "Energie" in thead.get_text():
                Fuel.append(tbody.get_text().strip())
            elif "Puissance fiscale" in thead.get_text():
                Horse_Power.append( tbody.get_text().strip())
            elif "Boîte" in thead.get_text():
                Gear_box.append(tbody.get_text().strip())
            elif "Transmission" in thead.get_text():
                Transmission.append (tbody.get_text().strip())
            elif "Couleur" in thead.get_text():
                Color.append(tbody.get_text().strip())

    

In [10]:
Fuel.append('Essence')

In [11]:
Cars_data={
    'Brand': Brand,
    'Type': Type,
    'Manufacture_year': Manufacture_year,
    'Mileage': Mileage,
    'Price': Price,
    'Body_type': Body_type,
    'Fuel': Fuel,
    'Horse_Power': Horse_Power,
    'Gear_box': Gear_box,
    'Transmission': Transmission,
    'Color': Color,
    'Governorate': Governorate,
    'Address': Address
}

In [12]:
df=pd.DataFrame(Cars_data)

In [13]:
df.to_csv('CarsData.csv',index=False)